In [38]:
import os
import sys
import json
import pickle
import torch

from preprocess.common_utils import quote_normalization

In [37]:
!python --version

Python 3.6.5 :: Anaconda, Inc.


## Data inspection

In [19]:
dev_preproc_path = 'data/dev.bin'
with open(dev_preproc_path, 'rb') as f:
    dev_preproc_dataset = pickle.load(f)
len(dev_preproc_dataset)

1034

In [20]:
dev_preproc_dataset[0]

{'db_id': 'concert_singer',
 'query': 'SELECT count(*) FROM singer',
 'query_toks': ['SELECT', 'count', '(', '*', ')', 'FROM', 'singer'],
 'query_toks_no_value': ['select', 'count', '(', '*', ')', 'from', 'singer'],
 'question': 'How many singers do we have?',
 'question_toks': ['How', 'many', 'singers', 'do', 'we', 'have', '?'],
 'sql': {'except': None,
  'from': {'conds': [], 'table_units': [['table_unit', 1]]},
  'groupBy': [],
  'having': [],
  'intersect': None,
  'limit': None,
  'orderBy': [],
  'select': [False, [[3, [0, [0, 0, False], None]]]],
  'union': None,
  'where': []},
 'raw_question_toks': ['how', 'many', 'singers', 'do', 'we', 'have', '?'],
 'processed_question_toks': ['how', 'many', 'singer', 'do', 'we', 'have', '?'],
 'pos_tags': ['WRB', 'JJ', 'NNS', 'VBP', 'PRP', 'VB', '.'],
 'relations': [['question-question-identity',
   'question-question-dist1',
   'question-question-dist2',
   'question-question-generic',
   'question-question-generic',
   'question-question-

In [23]:
dev_preproc_dataset[0].keys()

dict_keys(['db_id', 'query', 'query_toks', 'query_toks_no_value', 'question', 'question_toks', 'sql', 'raw_question_toks', 'processed_question_toks', 'pos_tags', 'relations', 'schema_linking', 'used_tables', 'used_columns', 'ast', 'actions'])

In [21]:
dev_lgesql_path = 'data/dev.lgesql.bin'
with open(dev_lgesql_path, 'rb') as f:
    dev_lgesql_dataset = pickle.load(f)
len(dev_lgesql_dataset)

Using backend: pytorch
/home/yshao/anaconda3/envs/lgesql/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


1034

In [24]:
dev_lgesql_dataset[0].keys()

dict_keys(['db_id', 'query', 'query_toks', 'query_toks_no_value', 'question', 'question_toks', 'sql', 'raw_question_toks', 'processed_question_toks', 'pos_tags', 'relations', 'schema_linking', 'used_tables', 'used_columns', 'ast', 'actions', 'graph'])

In [27]:
dev_lgesql_dataset[0]['graph']

In [28]:
dev_lgesql_dataset[0]['graph'].__dict__.keys()

dict_keys(['global_g', 'global_edges', 'local_g', 'local_edges', 'question_mask', 'schema_mask', 'gp', 'node_label', 'lg'])

In [33]:
_g_edges = dev_lgesql_dataset[0]['graph'].global_edges
_g_edges

[(0, 1, 'question-question-dist1'),
 (0, 7, 'question-table-nomatch'),
 (0, 8, 'question-table-nomatch'),
 (0, 9, 'question-table-nomatch'),
 (0, 10, 'question-table-nomatch'),
 (0, 11, 'question-column-nomatch'),
 (0, 12, 'question-column-nomatch'),
 (0, 13, 'question-column-nomatch'),
 (0, 14, 'question-column-nomatch'),
 (0, 15, 'question-column-nomatch'),
 (0, 16, 'question-column-nomatch'),
 (0, 17, 'question-column-nomatch'),
 (0, 18, 'question-column-nomatch'),
 (0, 19, 'question-column-nomatch'),
 (0, 20, 'question-column-nomatch'),
 (0, 21, 'question-column-nomatch'),
 (0, 22, 'question-column-nomatch'),
 (0, 23, 'question-column-nomatch'),
 (0, 24, 'question-column-nomatch'),
 (0, 25, 'question-column-nomatch'),
 (0, 26, 'question-column-nomatch'),
 (0, 27, 'question-column-nomatch'),
 (0, 28, 'question-column-nomatch'),
 (0, 29, 'question-column-nomatch'),
 (0, 30, 'question-column-nomatch'),
 (0, 31, 'question-column-nomatch'),
 (0, 32, 'question-column-nomatch'),
 (1, 0, '

In [34]:
_l_edges = dev_lgesql_dataset[0]['graph'].local_edges
_l_edges

[(0, 1, 'question-question-dist1'),
 (0, 7, 'question-table-nomatch'),
 (0, 8, 'question-table-nomatch'),
 (0, 9, 'question-table-nomatch'),
 (0, 10, 'question-table-nomatch'),
 (0, 11, 'question-column-nomatch'),
 (0, 12, 'question-column-nomatch'),
 (0, 13, 'question-column-nomatch'),
 (0, 14, 'question-column-nomatch'),
 (0, 15, 'question-column-nomatch'),
 (0, 16, 'question-column-nomatch'),
 (0, 17, 'question-column-nomatch'),
 (0, 18, 'question-column-nomatch'),
 (0, 19, 'question-column-nomatch'),
 (0, 20, 'question-column-nomatch'),
 (0, 21, 'question-column-nomatch'),
 (0, 22, 'question-column-nomatch'),
 (0, 23, 'question-column-nomatch'),
 (0, 24, 'question-column-nomatch'),
 (0, 25, 'question-column-nomatch'),
 (0, 26, 'question-column-nomatch'),
 (0, 27, 'question-column-nomatch'),
 (0, 28, 'question-column-nomatch'),
 (0, 29, 'question-column-nomatch'),
 (0, 30, 'question-column-nomatch'),
 (0, 31, 'question-column-nomatch'),
 (0, 32, 'question-column-nomatch'),
 (1, 0, '

In [36]:
set(_l_edges) - set(_g_edges)

set()

In [35]:
for e in _g_edges:
    if e not in _l_edges:
        print(e)

(0, 0, 'question-question-identity')
(0, 2, 'question-question-dist2')
(0, 3, 'question-question-generic')
(0, 4, 'question-question-generic')
(0, 5, 'question-question-generic')
(0, 6, 'question-question-generic')
(1, 1, 'question-question-identity')
(1, 3, 'question-question-dist2')
(1, 4, 'question-question-generic')
(1, 5, 'question-question-generic')
(1, 6, 'question-question-generic')
(2, 0, 'question-question-dist-2')
(2, 2, 'question-question-identity')
(2, 4, 'question-question-dist2')
(2, 5, 'question-question-generic')
(2, 6, 'question-question-generic')
(3, 0, 'question-question-generic')
(3, 1, 'question-question-dist-2')
(3, 3, 'question-question-identity')
(3, 5, 'question-question-dist2')
(3, 6, 'question-question-generic')
(4, 0, 'question-question-generic')
(4, 1, 'question-question-generic')
(4, 2, 'question-question-dist-2')
(4, 4, 'question-question-identity')
(4, 6, 'question-question-dist2')
(5, 0, 'question-question-generic')
(5, 1, 'question-question-generic')


## Temp

In [12]:
q = "What is the checking balance of the account whose owner\u2019s name contains the substring \u2018ee\u2019?"
q

'What is the checking balance of the account whose owner’s name contains the substring ‘ee’?'

In [13]:
print('Question:', q)

Question: What is the checking balance of the account whose owner’s name contains the substring ‘ee’?


In [17]:
q_toks = [
    "What",
    "is",
    "the",
    "checking",
    "balance",
    "of",
    "the",
    "account",
    "whose",
    "owner\u2019s",
    "name",
    "contains",
    "the",
    "substring",
    "\u2018ee\u2019",
    "?"]
quote_normalization(q_toks)

['What',
 'is',
 'the',
 'checking',
 'balance',
 'of',
 'the',
 'account',
 'whose',
 'owner’s',
 'name',
 'contains',
 'the',
 'substring',
 '"',
 'ee',
 '"',
 '?']

In [7]:
q.encode('ascii')

UnicodeEncodeError: 'ascii' codec can't encode character '\u2018' in position 65: ordinal not in range(128)

In [11]:
s = 'ç'
s.encode('ascii')

UnicodeEncodeError: 'ascii' codec can't encode character '\xe7' in position 0: ordinal not in range(128)

### Test functions

In [58]:
A = torch.zeros((10, 5), dtype=int)
mask = torch.BoolTensor([0, 0, 1, 0, 0, 1, 1, 0, 0, 1]).unsqueeze(-1)
src = torch.eye(5, dtype=int)
A, mask, src

(tensor([[0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0]]),
 tensor([[False],
         [False],
         [ True],
         [False],
         [False],
         [ True],
         [ True],
         [False],
         [False],
         [ True]]),
 tensor([[1, 0, 0, 0, 0],
         [0, 1, 0, 0, 0],
         [0, 0, 1, 0, 0],
         [0, 0, 0, 1, 0],
         [0, 0, 0, 0, 1]]))

In [59]:
torch.masked_scatter(A, mask, src)

tensor([[0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0],
        [1, 0, 0, 0, 0],
        [0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0],
        [0, 1, 0, 0, 0],
        [0, 0, 1, 0, 0],
        [0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0],
        [0, 0, 0, 1, 0]])